## EDA:
- **Principais Insights**:
    - O dataset é bem simplista, me limitando bastante de fazer analises mais profundas e precisas
    - Tudo bem, pois o foco do projeto não é a EDA em si, mas a modelagem de elasticidade-preço
    - Aqui encontrará uma analise simples da estrutura do dataset e alguns calculos de elasticidade-preço por categoria

---

In [1]:
from src.data.data_ingestion import IngestionFactory, DataIngestionContext
import numpy as np
import os
from dotenv import load_dotenv

from src.utils.elasticidade import calculate_elasticity
from src.utils.outlier_detection import outlier_detection


In [2]:

db_params = {
    "database": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
}

load_dotenv()

strategy = IngestionFactory.create("all")
context = DataIngestionContext(db_params, strategy)
df = context.execute()

C:\Users\compu\PycharmProjects\ElasticidadeMachineLearning\src\data_ingestion.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


### Visão Geral

In [3]:
df.head()

,id,date,customer_id,transaction_id,sku_category,sku,quantity,sales_amount
0,0cfb7a14-fe4d-4ef4-8261-a924dce63830,2016-01-02,2547,1,X52,0EM7L,1.0,3.13
1,844e4eb3-eff0-4051-9751-94dfc328daae,2016-01-02,822,2,2ML,68BRQ,1.0,5.46
2,a7ac73f8-435b-4318-8eae-14d6d5fe572b,2016-01-02,3686,3,0H2,CZUZX,1.0,6.35
3,469c5bd4-4a0c-4b8a-aec1-6c3c0a1ed6bf,2016-01-02,3719,4,0H2,549KK,1.0,5.59
4,4cf5d2a9-f6d0-4eec-bcb1-e5bf3a8989c0,2016-01-02,9200,5,0H2,K8EHH,1.0,6.88


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131706 entries, 0 to 131705
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              131706 non-null  object 
 1   date            131706 non-null  object 
 2   customer_id     131706 non-null  object 
 3   transaction_id  131706 non-null  object 
 4   sku_category    131706 non-null  object 
 5   sku             131706 non-null  object 
 6   quantity        131706 non-null  float64
 7   sales_amount    131706 non-null  float64
dtypes: float64(2), object(6)
memory usage: 8.0+ MB


In [5]:
df.describe()

,quantity,sales_amount
count,131706.000000,131706.000000
mean,1.485311,11.981524
std,3.872667,19.359699
min,0.010000,0.020000
25%,1.000000,4.230000
50%,1.000000,6.920000
75%,1.000000,12.330000
max,400.000000,707.730000


In [6]:
df.isnull().sum()

id                0
date              0
customer_id       0
transaction_id    0
sku_category      0
sku               0
quantity          0
sales_amount      0
dtype: int64

In [7]:
df.duplicated().sum()

0

In [8]:
df.nunique()

id                131706
date                 363
customer_id        22625
transaction_id     64682
sku_category         187
sku                 5242
quantity              83
sales_amount        3233
dtype: int64

In [9]:
df.select_dtypes(exclude=[np.number]).apply(lambda col: col.astype(str).str.contains("NaN").sum())

id                0
date              0
customer_id       0
transaction_id    0
sku_category      0
sku               0
dtype: int64

In [10]:
numeric_cols = df.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    outliers = outlier_detection(df, col)
    print(f"\noutliers na coluna: {col}")
    print(outliers[[col]].head())



outliers na coluna: quantity
    quantity
20       2.0
42       2.0
48       2.0
74       3.0
93       2.0

outliers na coluna: sales_amount
     sales_amount
60          26.57
70          30.00
125        118.35
128         33.60
145         24.93


#### Insights Infrastructure :
- **Business**:
    - 187 categorias diferentes, mais de 22 mil clientes cadastrados, mais de 5mil produtos presentes
    - Para descobrir o preço unitario do item é necessário realizar a divisão entre sales_amount e quantity
- **Features**:
    - Criação de nova feature para preço unitário
    - Por ser um dataset kaggle, existe alguns gaps de negócio nesses dados, como falta de descrição, outras tabelas complementares, moeda negociada, escala de preços, etc.
    - Percebi que a EDA aqui será bem superficial, não tem problema pois o foco do projeto é a modelagem de elasticidade-preço demanda

### Elasticidade

In [11]:
df['price_unit'] = df['sales_amount'] / df['quantity']

In [34]:
elasticidade_category = calculate_elasticity(df, group_by=["sku_category"])
elasticidade_category_p_value = elasticidade_category[elasticidade_category['p_value'] < 0.05]
category_p_value = (elasticidade_category_p_value['group'].nunique() / df['sku_category'].nunique()) * 100
category_p_value

C:\Users\compu\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\compu\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\compu\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\compu\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\compu\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\compu\anaconda3\Lib\site-packages\statsmodels\regressio

47.593582887700535

In [35]:
elasticidade_category = calculate_elasticity(df, group_by=["sku"])
elasticidade_category_p_value = elasticidade_category[elasticidade_category['p_value'] < 0.05]
category_p_value = (elasticidade_category_p_value['group'].nunique() / df['sku'].nunique()) * 100
category_p_value

C:\Users\compu\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\compu\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\compu\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\compu\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\compu\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\compu\anaconda3\Lib\site-packages\statsmodels\regression

18.351774132010686

Insights:
- ~47.59% das categorias e ~18.35% tem p_value significativo, indicando certa confiabilidade nas informações aresentadas de elasticity